# Extracting retraction reasons with the OpenAI API

Proof of concept

TODO: evaluate model performance, select a model (considering pricing too)

In [2]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [8]:
import openai
import os
import pandas as pd
import json
import re
import numpy as np

In [24]:
MODEL = "text-davinci-001"

In [16]:
filepath = "../../../data/ePMC/retracts.json"
with open(filepath, 'r') as fp:
    data = json.load(fp)

df = pd.DataFrame.from_dict(data)
retrieved = df.dropna()


In [22]:
with open('assets/openAI_key.txt', 'r') as f:
    api_key = f.read().strip()

os.environ['OPENAI_API_KEY'] = api_key

openai.api_key = os.getenv("OPENAI_API_KEY")

In [17]:
# Define the list of retraction notices
notices = [
    "invalid results",
    "plagiarism",
    "conflict of interest, legal reasons",
    "research misconduct and data manipulation",
    "agreement by author(s)",
    "reader concerns",
    "issues with authorship",
    "duplicated paper",
    "could not reproduce results",
] 

# Define a dictionary to store the results
results = {reason:[] for reason in notices}

# Define the prompt to extract retraction reasons
reasons_string = "\n-".join(results.keys())
reason_prompt = f"Given a list of possible reasons for retraction, please extract the reason(s) for retraction from a given text and return a comma-separated string of the identified reasons. The list of possible reasons is as follows: \n-{reasons_string} \nFor example, if the input text is: \"The article is suspected to contain manipulated data (...) The authors agreed to retract the article.\", the output should be \"research misconduct and data manipulation, agreement by author(s)\". There must not be any sentence in the output phrased differently than the provided retraction reason list."

print(results)

{'invalid results': [], 'plagiarism': [], 'conflict of interest, legal reasons': [], 'research misconduct and data manipulation': [], 'agreement by author(s)': [], 'reader concerns': [], 'issues with authorship': [], 'duplicated paper': [], 'could not reproduce results': []}


In [27]:
# Function to loop over the retraction notices and request an OpenAI model its retraction reason(s)
def get_openAI_retraction(data, model, i=10):
    tokens = 0
    llm_reasons_unfiltered = {}
    llm_reasons_filtered = {}
    for index, row in data.iterrows(): 
        i+=1
        if i<10: #limit to 10 to keep it cheap for now
            notice = row["fullText"]
            #print(notice)
            #id = 
            word_count = len(notice.split())
            tokens += word_count
            #print(f"Article with identifier {id} has a length of {word_count}  characters, approximately {word_count*1000/750} tokens. ~{tokens}    tokens used so far")
            reason_text = reason_prompt + " " + notice
            reason_result = openai.Completion.create(
                model=model,
                prompt=reason_text,
                max_tokens=1024,
                stop=None,
                temperature=0.0,# As deterministic as possible
            )
            reasons_string = reason_result["choices"][0]["text"].strip()
            #print(reasons_string)
            reasons_filtered = []
            for reason in notices: 
                if reason in reasons_string:
                    reasons_filtered.append(reason)

            llm_reasons_unfiltered[id] = reasons_string.split(", ")
            llm_reasons_filtered[id] = reasons_filtered
            data.at[index, "llm-reason"] = ", ".join(reasons_filtered)
    return data
        



In [36]:
davinci_one = get_openAI_retraction(retrieved, "text-davinci-001").dropna()
davinci_two = get_openAI_retraction(retrieved, "text-davinci-002").dropna()
davinci_three = get_openAI_retraction(retrieved, "text-davinci-003").dropna()
curie_one = get_openAI_retraction(retrieved, "text-curie-001").dropna()

In [45]:
see_data = {}
models = ["text-davinci-001", "text-davinci-002", "text-davinci-003", "text-curie-001"]
for i in range(0,8):
    row = {}
    id = retrieved.loc[retrieved.index[i], 'source'] + retrieved.loc[retrieved.index[i], 'id']
    row["fullText"] = retrieved.loc[retrieved.index[i],'fullText']
    row["text-davinci-001"] = davinci_one.loc[davinci_one.index[i], "llm-reason"]
    row["text-davinci-002"] = davinci_two.loc[davinci_two.index[i], "llm-reason"]
    row["text-davinci-003"] = davinci_three.loc[davinci_three.index[i], "llm-reason"]
    row["text-curie-001"] = curie_one.loc[curie_one.index[i], "llm-reason"]
    see_data[id] = row


llm_summary = pd.DataFrame.from_dict(see_data).transpose()
llm_summary.to_csv("test_llm.csv")